<a href="https://colab.research.google.com/github/polly-yu/llm_emotion/blob/main/experiment2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [4]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import transformers
from sentence_transformers import util

In [12]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [8]:
result_df_filtered = pd.read_csv('pairs.csv')
result_df_filtered.tail()

,person1_id,person1_name,person2_id,person2_name,emotion1,emotion2,cos,Is_diff_emotion,Is_diff_person,person1_gender,person2_gender,person1_age,person2_age,Is_diff_gender,Is_diff_age_group
4027,3,Женщина,3,Женщина,разочарование,презрение,0.743977,1,0,Female,Female,NaN,NaN,0.0,NaN
4028,3,Женщина,3,Женщина,разочарование,агрессия,0.556850,1,0,Female,Female,NaN,NaN,0.0,NaN
4029,3,Женщина,3,Женщина,раскаяние,презрение,0.578382,1,0,Female,Female,NaN,NaN,0.0,NaN
4030,3,Женщина,3,Женщина,раскаяние,агрессия,0.478956,1,0,Female,Female,NaN,NaN,0.0,NaN
4031,3,Женщина,3,Женщина,презрение,агрессия,0.560791,1,0,Female,Female,NaN,NaN,0.0,NaN


In [9]:
# cos_sim_personas

# Межперсональные значения (вне диагонали)
df_diff_person = result_df_filtered[result_df_filtered['Is_diff_person'] == 1].copy()
df_diff_person['person_pair'] = df_diff_person.apply(
    lambda row: tuple(sorted([row['person1_name'], row['person2_name']])), axis=1
)

agg_offdiag = df_diff_person.groupby('person_pair')['cos'].mean().reset_index()
agg_offdiag[['person1', 'person2']] = pd.DataFrame(agg_offdiag['person_pair'].tolist(), index=agg_offdiag.index)
agg_offdiag = agg_offdiag[['person1', 'person2', 'cos']]
agg_offdiag.columns = ['person1', 'person2', 'mean_CS']

# Внутриперсональные значения (диагональ)
df_same_person = result_df_filtered[result_df_filtered['Is_diff_person'] == 0].copy()
agg_diag = df_same_person.groupby('person1_name')['cos'].mean().reset_index()
agg_diag['person2'] = agg_diag['person1_name']
agg_diag = agg_diag[['person1_name', 'person2', 'cos']]
agg_diag.columns = ['person1', 'person2', 'mean_CS']

# Объединяем всё
agg_all = pd.concat([agg_offdiag, agg_diag], ignore_index=True)

# Строим симметричную матрицу для нужного порядка групп
desired_order = ["Ребенок", "Взрослый", "Мужчина", "Женщина"]
matrix = agg_all.pivot(index='person1', columns='person2', values='mean_CS')
matrix_full = matrix.combine_first(matrix.T)
matrix_full = matrix_full.loc[desired_order, desired_order]

print(matrix_full.round(3))


          Ребенок  Взрослый  Мужчина  Женщина
person1                                      
Ребенок     0.653     0.632      NaN      NaN
Взрослый    0.632     0.632      NaN      NaN
Мужчина       NaN       NaN    0.621    0.618
Женщина       NaN       NaN    0.618    0.607


In [10]:
# Проверка стандартного отклонения по всем косинусным схожестям между персонами
print(result_df_filtered.groupby(['person1_name', 'person2_name'])['cos'].agg(['mean', 'std']).sort_values('std').round(3))

                            mean    std
person1_name person2_name              
Взрослый     Взрослый      0.632  0.083
Женщина      Женщина       0.607  0.085
Мужчина      Мужчина       0.621  0.086
Ребенок      Взрослый      0.632  0.093
             Ребенок       0.653  0.098
Мужчина      Женщина       0.618  0.102


In [19]:
emot_names = ['гнев', 'изумление', 'отвращение', 'ужас', 'восхищение', 'восторг', 'горе', 'настороженность',
              'доверие', 'печаль', 'ожидание', 'злость', 'неудовольствие', 'радость', 'страх', 'удивление',
              'интерес', 'безмятежность', 'принятие', 'тревога', 'растерянность', 'скука', 'досада', 'грусть',
              'любовь', 'оптимизм', 'покорность', 'трепет', 'разочарование', 'раскаяние', 'презрение', 'агрессия']

In [22]:
#cos_sim_emotions

# Межэмоциональные значения (вне диагонали)
df_diff_emotion = result_df_filtered[result_df_filtered['Is_diff_emotion'] == 1].copy()
df_diff_emotion['emotion_pair'] = df_diff_emotion.apply(
    lambda row: tuple(sorted([row['emotion1'], row['emotion2']])), axis=1
)

# Рассчитаем среднее и std для межэмоциональных пар
agg_offdiag_emotion_stats = df_diff_emotion.groupby('emotion_pair')['cos'].agg(['mean', 'std']).reset_index()

# Распаковываем пары в отдельные колонки
agg_offdiag_emotion_stats[['emotion1', 'emotion2']] = pd.DataFrame(agg_offdiag_emotion_stats['emotion_pair'].tolist(), index=agg_offdiag_emotion_stats.index)

# Внутремоциональные значения (диагональ)
df_same_emotion = result_df_filtered[result_df_filtered['Is_diff_emotion'] == 0].copy()
agg_diag_emotion_stats = df_same_emotion.groupby('emotion1')['cos'].agg(['mean', 'std']).reset_index()
agg_diag_emotion_stats['emotion2'] = agg_diag_emotion_stats['emotion1']

# Объединяем
agg_diag_emotion_stats = agg_diag_emotion_stats[['emotion1', 'emotion2', 'mean', 'std']]
agg_offdiag_emotion_stats = agg_offdiag_emotion_stats[['emotion1', 'emotion2', 'mean', 'std']]

agg_all_emotion_stats = pd.concat([agg_offdiag_emotion_stats, agg_diag_emotion_stats], ignore_index=True)

# Пивотируем средние и std в две матрицы
mean_matrix = agg_all_emotion_stats.pivot(index='emotion1', columns='emotion2', values='mean')
std_matrix = agg_all_emotion_stats.pivot(index='emotion1', columns='emotion2', values='std')

# Заполняем симметричные пропуски
mean_matrix_full = mean_matrix.combine_first(mean_matrix.T)
std_matrix_full = std_matrix.combine_first(std_matrix.T)

mean_matrix_full.round(3).to_csv('cos_sim_emot_mean.csv')
std_matrix_full.round(3).to_csv('cos_sim_emot_std.csv')

# Упорядочим по желаемому списку эмоций
desired_order_emotions = ['гнев', 'изумление', 'отвращение', 'ужас', 'восхищение']

mean_matrix_full = mean_matrix_full.loc[desired_order_emotions, desired_order_emotions]
std_matrix_full = std_matrix_full.loc[desired_order_emotions, desired_order_emotions]

print("Средние косинусные сходства:\n", mean_matrix_full.round(3))
print("\nСтандартные отклонения:\n", std_matrix_full.round(3))


Средние косинусные сходства:
              гнев  изумление  отвращение   ужас  восхищение
emotion1                                                   
гнев        0.687      0.574       0.526  0.641       0.584
изумление   0.574      0.834       0.574  0.618       0.768
отвращение  0.526      0.574       0.881  0.598       0.554
ужас        0.641      0.618       0.598  0.807       0.612
восхищение  0.584      0.768       0.554  0.612       0.793

Стандартные отклонения:
              гнев  изумление  отвращение   ужас  восхищение
emotion1                                                   
гнев        0.082      0.066       0.056  0.056       0.046
изумление   0.066      0.160       0.064  0.037       0.055
отвращение  0.056      0.064       0.161  0.041       0.044
ужас        0.056      0.037       0.041  0.073       0.038
восхищение  0.046      0.055       0.044  0.038       0.152


In [13]:
# однофакторная ANOVA: Is_diff_emotion
model_emotion = ols('cos ~ C(Is_diff_emotion)', data=result_df_filtered).fit()
anova_emotion = sm.stats.anova_lm(model_emotion, typ=2)
print(anova_emotion)

                       sum_sq      df           F         PR(>F)
C(Is_diff_emotion)   3.940351     1.0  503.231877  4.012391e-105
Residual            31.555267  4030.0         NaN            NaN


In [15]:
result_df_filtered.groupby('Is_diff_emotion')['cos'].describe()

,count,mean,std,min,25%,50%,75%,max
Is_diff_emotion,,,,,,,,
0,64.0,0.872855,0.096016,0.629355,0.818995,0.875930,0.952260,1.000000
1,3968.0,0.622733,0.088363,0.370058,0.563777,0.616398,0.671945,0.975965


In [16]:
# однофакторная ANOVA: Is_diff_gender
model_gender = ols('cos ~ C(Is_diff_gender)', data=result_df_filtered).fit()
anova_gender = sm.stats.anova_lm(model_gender, typ=2)
print(anova_gender)

                      sum_sq      df         F    PR(>F)
C(Is_diff_gender)   0.008480     1.0  0.958003  0.327808
Residual           17.826739  2014.0       NaN       NaN


In [17]:
# однофакторная ANOVA: Is_diff_age_group
model_age_group = ols('cos ~ C(Is_diff_age_group)', data=result_df_filtered).fit()
anova_age_group = sm.stats.anova_lm(model_age_group, typ=2)
print(anova_age_group)

                         sum_sq      df         F   PR(>F)
C(Is_diff_age_group)   0.054878     1.0  6.443657  0.01121
Residual              17.152548  2014.0       NaN      NaN
